<a href="https://colab.research.google.com/github/mathluva/Bert-Toxic-Comments/blob/main/Toxic_Comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Feb 28 04:52:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import files
uploaded = files.upload()

Saving train_preprocessed.csv to train_preprocessed.csv


In [172]:
import io
df = pd.read_csv(io.BytesIO(uploaded['train_preprocessed.csv']))
df.head()

,comment_text,id,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity
0,explanation why the edits made under my userna...,0000997932d777bf,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
1,d aww he matches this background colour i m s...,000103f0d9cfb60f,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
2,hey man i m really not trying to edit war it...,000113f07ec002fd,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
3,more i can t make any real suggestions on im...,0001b41b1c6bb37e,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
4,you sir are my hero any chance you remember...,0001d958c54c6e35,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0


In [173]:
rows = len(df)
print(f'There are {rows} rows')

There are 159571 rows


In [174]:
ids = df.id.nunique()
print(f'There are {ids} unique ids')

There are 159571 unique ids


In [175]:
#replace index with id
df.set_index('id', inplace =True) 

In [176]:
#toxicity is total of all categories per comment
# toxicity 6 means comment was classified for all categories
df.toxicity.value_counts() 

0.0    143346
1.0      6360
3.0      4209
2.0      3480
4.0      1760
5.0       385
6.0        31
Name: toxicity, dtype: int64

In [177]:
#keep comments with only one label
df = df[df['toxicity'].values ==1]
df

,comment_text,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity
id,,,,,,,,,
0005c987bdfc9d4b,hey what is it talk what is it an exclusive ...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
0007e25b2121310b,bye don t look come or think of comming back...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
001956c382006abd,i m sorry i m sorry i screwed around with some...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
002264ea4d5f2887,why can t you believe how fat artie is did yo...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
005f59485fcddeb0,sorry puck but no one ever said dick was numb...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
fcf5a6ad5918f164,your boring and retard ed two,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
fd0129fde97321cb,why did that idiot revert the reversion i made...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0
fd68ef478b3dfd05,ps you re all middle aged losers at home in yo...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0


In [178]:
#check sum of identity_hate
df.identity_hate.value_counts()

0.0    6306
1.0      54
Name: identity_hate, dtype: int64

In [179]:
#create new column label
#decode the rows
for index, row in df.iterrows():
    if df.loc[index, 'identity_hate'] == 1:
      df.loc[index, 'label'] = 'identity_hate'

    elif df.loc[index, 'insult'] == 1:
      df.loc[index, 'label'] = 'insult'

    elif df.loc[index, 'obscene'] == 1:
      df.loc[index, 'label'] = 'obscene'

    elif df.loc[index, 'severe_toxic'] == 1:
      df.loc[index, 'label'] = 'severe_toxic'

    elif df.loc[index, 'threat'] == 1:
      df.loc[index, 'label'] = 'threat'

    else:
        df.loc[index, 'label'] = 'toxic'

In [180]:
#create new column to hold label value
possible_lables = ['identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic']

label_dict = {}
for index, label in enumerate(possible_labels):
    label_dict[label] = index

df['label_value'] = df.label.replace(label_dict)

In [181]:
df.tail(20)

,comment_text,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity,label,label_value
id,,,,,,,,,,,
f776500f8c82b957,he s just left handed and left handed people ...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
f7fa6643da4f8c70,down with the usa,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
f831fd421d7b6e44,you read it your note addressed something els...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
f896893958bb1cec,oh dear oh dear how fuck in sad i will regre...,0.0,0.0,1.0,train,0.0,0.0,0.0,1.0,obscene,2
f8f20423b1f45a33,get lost get lost steve and take your purile t...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
f90ba6ec104fceee,he wouldn t have to if he weren t so goddamn c...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
f92bb6c394ccf1b4,hey queer boy stop messing with my edits,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
f97edebe2f9dc0db,it s just the damn internet,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5
f9e9180ab9469207,naughty sockpuppet very naughty indeed i sugg...,0.0,0.0,0.0,train,0.0,0.0,1.0,1.0,toxic,5


In [182]:
#check totals
df['label'].value_counts()

toxic            5666
obscene           317
insult            301
identity_hate      54
threat             22
Name: label, dtype: int64

In [183]:
df = df.drop(columns= ['set', 'toxicity'])
df.head()

,comment_text,identity_hate,insult,obscene,severe_toxic,threat,toxic,label,label_value
id,,,,,,,,,
0005c987bdfc9d4b,hey what is it talk what is it an exclusive ...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5
0007e25b2121310b,bye don t look come or think of comming back...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5
001956c382006abd,i m sorry i m sorry i screwed around with some...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5
002264ea4d5f2887,why can t you believe how fat artie is did yo...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5
005f59485fcddeb0,sorry puck but no one ever said dick was numb...,0.0,0.0,0.0,0.0,0.0,1.0,toxic,5


In [184]:
#use stratified sampling to ensure that sampled dataset is representative of the entire population
X_train, X_val, y_train, y_val= train_test_split(df.index.values, df.label_value.values, random_state = 42, stratify = df['label'].values)


In [185]:
#create set column
#locate training and val samples
df['sample_type'] = ['none']*df.shape[0]

df.loc[X_train, 'sample_type'] = 'train'
df.loc[X_val, 'sample_type'] = 'val'

In [187]:
#check stratified sampling values

df.groupby(['label', 'label_value', 'sample_type']).count()

comment_text  ...  toxic
label         label_value sample_type                ...       
identity_hate 0           train                  40  ...     40
                          val                    14  ...     14
insult        1           train                 226  ...    226
                          val                    75  ...     75
obscene       2           train                 238  ...    238
                          val                    79  ...     79
threat        4           train                  17  ...     17
                          val                     5  ...      5
toxic         5           train                4249  ...   4249
                          val                  1417  ...   1417

[10 rows x 7 columns]

In [189]:
!pip install transformers==3.5

     |████████████████████████████████| 1.3MB 8.3MB/s 
     |████████████████████████████████| 1.1MB 52.7MB/s 
     |████████████████████████████████| 890kB 55.7MB/s 
     |████████████████████████████████| 2.9MB 55.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=39c1951138e0e74293319aaa85a9a97cefe9ffd8dd2fa84815d060081be56d49
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [190]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [191]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)


In [198]:
#encode train data
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.sample_type=='train'].comment_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    max_length = 256,
    padding = True,
    truncation = True,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.sample_type=='train'].label_value.values)



In [199]:
#encode validation data
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.sample_type=='val'].comment_text.values, 
   add_special_tokens=True, 
    return_attention_mask=True, 
    max_length = 256,
    padding = True,
    truncation = True,
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.sample_type=='val'].label_value.values)

In [201]:
# Convert all inputs and labels into torch tensors
#BERT model required datatype 
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [203]:
print(len(dataset_train))
print(len(dataset_val))

4770
1590


In [204]:
from transformers import BertForSequenceClassification

In [205]:
 model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                       num_labels=len(possible_labels),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [206]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [207]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [208]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [209]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [210]:
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [211]:
from sklearn.metrics import f1_score

In [212]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [213]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [214]:
import random
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [215]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [216]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [217]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.5808563445011775
Validation loss: 0.4512103407084942
F1 Score (Weighted): 0.8399223614501173



Epoch 2
Training loss: 0.4419099797060092
Validation loss: 0.44641412913799283
F1 Score (Weighted): 0.8399223614501173



Epoch 3
Training loss: 0.42384224812189736
Validation loss: 0.43880823746323583
F1 Score (Weighted): 0.8399223614501173



In [218]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [219]:
model.to(torch.device('cuda'))

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [220]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [221]:
!ls /content/gdrive/My Drive

ls: cannot access '/content/gdrive/My': No such file or directory
ls: cannot access 'Drive': No such file or directory


In [222]:
model_save_name = 'classifier.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [226]:
model.load_state_dict(torch.load(path))


<All keys matched successfully>

In [227]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [228]:
accuracy_per_class(predictions, true_vals)

Class: identity_hate
Accuracy: 7/14

Class: insult
Accuracy: 2/75

Class: obscene
Accuracy: 5/79

Class: threat
Accuracy: 0/5

Class: toxic
Accuracy: 0/1417

